In [1]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
from plotly.graph_objs import Histogram as H
import plotly.plotly as py
import pandas as pd
import numpy as np
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import datetime
import seaborn as sns
from statsmodels.stats.proportion import proportion_confint
import numpy as np, scipy.stats as st

#init dataframes from csvs
df1 = pd.read_csv('../Data/amt240_allcd.csv') # amt df
df2 = pd.read_csv('../Data/dtu1_allcd.csv') # dtu1 df
df3 = pd.read_csv('../Data/dtu2_allcd.csv') # dtu2 df

In [2]:
#run to make barchart of "curse of knowledge" answers - DTU1 trial

df = df3

x_values = ['Arriving at 8:40', 'Arriving at 9:00']

# 8:400 answers
count = 0
total_count = len(df['cok840'].dropna())
percent_list1 = []
lower_error_bound = []
upper_error_bound = []

#find percent choosing canteen and percent choosing office
for i in [1., 0.]:
    count = len(df.loc[df['cok840'] == i])
    nob = total_count
    percent_list1.append(count / nob)
    lower, upper = proportion_confint(count, nob, alpha=0.05, method='normal')
    lower_error_bound.append(lower)
    upper_error_bound.append(upper)
#only one bound is needed, as its +/-
proper_lower_bound = []
for i in range(len(percent_list1)):
    proper_lower_bound.append(percent_list1[i] - lower_error_bound[i])

#now for 9:00
count2 = 0
total_count2 = len(df['cok900'].dropna())
percent_list2 = []
lower_error_bound2 = []
upper_error_bound2 = []

for i in [1., 0.]:
    count2 = len(df.loc[df['cok900'] == i])
    nob2 = total_count2
    percent_list2.append(count2 / nob2)
    
    lower2, upper2 = proportion_confint(count2, nob2, alpha=0.05, method='normal')
    lower_error_bound2.append(lower2)
    upper_error_bound2.append(upper2)
    
proper_lower_bound2 = []
for i in range(len(percent_list2)):
    proper_lower_bound2.append(percent_list2[i] - lower_error_bound2[i])
    
canteen_list = [percent_list1[0], percent_list2[0]]
office_list = [percent_list1[1], percent_list2[1]]
canteen_lower_error = [proper_lower_bound[0], proper_lower_bound2[0]]
office_lower_error = [proper_lower_bound[1], proper_lower_bound2[1]]

#barchart traces - generic names, only change data generation above to properly filter
trace1 = go.Bar(
    x=x_values,
    y=[percent_list1[0], percent_list2[0]],
    name = 'Canteen',
    error_y=dict(
        type='data',
        array=canteen_lower_error,
        visible=True
        ) 
    )
trace2 = go.Bar(
    x=x_values,
    y=[percent_list1[1], percent_list2[1]],
    name = 'Office',
    error_y=dict(
        type='data',
        array=office_lower_error,
        visible=True
        ) 
    )

data = [trace1, trace2]

#####################################
#Generating list of dictionaries for annotation variable,
#notice generic names, only change x_values and percent_list
key_list = [['x', 'y', 'xref', 'yref', 'text', 'showarrow', 'arrowhead', 'ax', 'ay', 'arrowsize']]*len(x_values)
my_list = []
#make values
value_list = []

for i in range(0, len(x_values)):
    my_list = []
    my_list.append(x_values[i]),
    my_list.append(percent_list1[i]),
    my_list.append('x'),
    my_list.append('y'),
    my_list.append(str(np.round(percent_list1, 3)[i])),
    my_list.append(True),
    my_list.append(7),
    #the next two items sets position
    my_list.append(-30),
    my_list.append(-30)
    my_list.append(0.75)
    value_list.append(my_list)
    
dict_list = []
for i in range(0, len(key_list)):
    a = key_list[i]
    b = value_list[i]
    my_tuple = tuple(zip(a,b))
    my_dict = {}
    for key, value in my_tuple:
        my_dict[key] = value
    dict_list.append(my_dict)
###############################


layout = go.Layout(
    barmode = 'group',
    #annotations = dict_list,
    title = '[DTU1] Barchart for Curse of Knowledge question. <br> You arrive at x, you secretly know your colleague arrived at 8:50 <br> Where does your colleague go?',
    font=dict(
        family='Open Sans, monospace', #Courier New
        size=14,
        color='#000000'#'#7f7f7f'
    ),
    yaxis=dict(
        title = 'Percent'
    ),
    xaxis=dict(
        title = 'Answers'
    )
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='plot7x_whose fault_barchart')
#barchart with error bars

C:\Users\Thomas\Miniconda3\envs\py37viz\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

